# Niwot Statistics Notebook
* t-tests on data with SW thresholds

#### Two-Sample T-Test: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
* using for Niwot and Laret Statistics

## Import packages 

In [1]:
# import packages 
%matplotlib widget
# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import csv 
import copy 
import os.path 

In [2]:
# stat packages, just for the stats notebooks
import scipy.stats as stats
from scipy.stats import norm # create normal distribution from dataset 
from scipy.stats import chisquare #Calculate a one-way chi-square test.
from scipy.stats import chi2_contingency #Chi-square test of independence of variables in a contingency table.

In [3]:
import seaborn as sns # a module that adds some plotting capabilities and makes your plots look better

sns.set() # activates some of the default settings from seaborn
# The following settings just set some defaults for the plots
plt.rcParams['figure.figsize']  = (12,4) #width, height
plt.rcParams['axes.titlesize']  = 14
plt.rcParams['axes.labelsize']  = 12
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 11
mpl.rcParams['figure.dpi'] = 100

sns.set_style("dark", {"xtick.bottom": True, 'ytick.left': True})

## Load and Clean datasets for use

In [4]:
# AMERIFLUX MET DATA 
niwotflux =   pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/niwot_2017_ameriflux_unload1hr.csv")
niwotflux['datetime']  = pd.DatetimeIndex(niwotflux['datetime'])
# niwotflux.index = pd.DatetimeIndex(niwotflux['datetime'])
# niwotflux.drop(columns=['datetime'], inplace=True)

# OBSERVATIONS CLASSIFICATIONS
niwotobs =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/datetimeformat_classifications_niwot2017.csv")
niwotobs['datetime'] = pd.to_datetime(niwotobs['datetime'])

# niwotobs.index = pd.DatetimeIndex(niwotobs['datetime'])
# niwotobs.drop(columns=['datetime'], inplace=True)

niwotobs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed 

# MERG THEM 
niwotdf = pd.merge(niwotflux, niwotobs, how='outer', on='datetime')
niwotdf.index = pd.DatetimeIndex(niwotdf['datetime'])

In [5]:
# Create symbols for sunlit or not
niwotdf['Esymbol'] = np.nan
# niwotdf['Eminus1symbol'] = np.nan

# open triangle for cloudy, diamond for sunny, open x circle for no radiation data
niwotdf['Esymbol'].mask(niwotdf['E'] == 0., 105, inplace=True) # this is working without fillna
niwotdf['Esymbol'].mask(niwotdf['E'] == 1., 2, inplace=True)  
# niwotdf['Eminus1symbol'].mask(niwotdf['Eminus1symbol'] == 0., 105, inplace=True) # this is working without fillna
# niwotdf['Eminus1symbol'].mask(niwotdf['Eminus1symbol'] == 1., 2, inplace=True)  

# then fillna with 128 for circle with x through it 
niwotdf['Esymbol'] = niwotdf['Esymbol'].fillna(128) 
# niwotdf['Eminus1symbol'] = niwotdf['Eminus1symbol'].fillna(128) 

In [6]:
# Create seperate df for only snow in the canopy timesteps 
df_unload = niwotdf.copy(deep=True)
df_unload = df_unload.dropna(axis=0, how='any', subset=['CD'])

# Have to remove all nans for this plotting to work.... need to come back to this 
df_unload.dropna(inplace=True) #########KEEP THIS IN MIND, COME BACK TO IT

# Create sunlit column
df_unload['Sunlit'] = df_unload.E.copy(deep=True)
df_unload.Sunlit.mask(df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
df_unload.Sunlit.mask(df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
df_unload['Classification'] = df_unload.CD.copy(deep=True)
df_unload.Classification.mask(df_unload.Classification == 0, "Snow Unloading", inplace=True)
df_unload.Classification.mask(df_unload.Classification == 1, "Snow Staying in the Canopy", inplace=True)

snowstaydf   = df_unload.where(df_unload.CD == 1).dropna() # where CD == 1, meaning Snow Staying, make that snowstaydf
snowunloaddf = df_unload.where(df_unload.CD == 0).dropna() # where CD == 0, meaning Snow Unloading, make that snowunloaddf

## Plotting Constants

In [7]:
## Define some plotting constants for easier coding 
plt.close('all')

# Colors
colornosnow = '#D2B48C' # nice tan
colorsnow = '#7dcfd4' # slightly desaturated cyan
colorsnowunload = '#1F15D5' # bright, deep blue 
colorsunny = '#E4E44A' # trying a little less bright 

# Names
namesnow = 'Snow Staying'
namesnowunload = 'Snow Unloading'
group_labels = ['Snow Staying', 'Snow Unloading']

nametemp = "Air Temperature (C)"
namewind = "Wind Speed (m/s)"
nameSW = "Shortwave (W/m2)"

# colors and names 
color1='tomato'; color2='maroon'; color3='navy'
colors = [color1, color2, color3]

# name1 = '600 < SW'; name2 = '400 < SW < 600'; name3 = '400 > SW'
name1 = 'SW > 600'; name2 = '400 < SW < 600'; name3 = 'SW < 400'
label = [name1, name2, name3]

## Make thresholded datasets

In [8]:
# Create copies of the df we want, and then going to classify it by SW threshold
dfSW600 =    df_unload.copy(deep=True) # SW > 600
dfSW400600 = df_unload.copy(deep=True) # SW between 400-600
dfSW400 =    df_unload.copy(deep=True) # SW < 400

# Create for temperature threshold 
dfTg0 =    df_unload.copy(deep=True) # Temp g(greater) 0
dfTl0 =    df_unload.copy(deep=True) # Temp l(less)    0 

# Creating df with only SW > 600
dfSW600.mask(dfSW600.shortwave < 600, inplace=True) # we want where SW > 600, else nan
dfSW600.dropna(inplace=True) # and drop all nan... 
dfSW600stay   = dfSW600.where(dfSW600.CD == 1).dropna()
dfSW600unload = dfSW600.where(dfSW600.CD == 0).dropna()

# Creating df with onyl 400 < SW < 600
dfSW400600.mask(dfSW400600.shortwave > 600, inplace=True) # we want where SW < 600, 
dfSW400600.mask(dfSW400600.shortwave < 400, inplace=True) # and SW > 400, else nan
dfSW400600.dropna(inplace=True) # and drop all nan... 
dfSW400600stay   = dfSW400600.where(dfSW400600.CD == 1).dropna()
dfSW400600unload = dfSW400600.where(dfSW400600.CD == 0).dropna()

# Creating df with SW < 400
dfSW400.mask(dfSW400.shortwave > 400, inplace=True) # we want where SW < 400, else nan
dfSW400.dropna(inplace=True) # and drop all nan...
dfSW400stay   = dfSW400.where(dfSW400.CD == 1).dropna()
dfSW400unload = dfSW400.where(dfSW400.CD == 0).dropna()

# Create df with T > 0 
dfTg0.mask(dfTg0.temp < 0, inplace=True) # we want where T > 0, else nan
dfTg0.dropna(inplace=True) 
dfTg0stay   = dfTg0.where(dfTg0.CD == 1).dropna()
dfTg0unload = dfTg0.where(dfTg0.CD == 0).dropna()

# Create df with T < 0 
dfTl0.mask(dfTl0.temp > 0, inplace=True) # we want where T < 0, else nan 
dfTl0.dropna(inplace=True)
dfTl0stay   = dfTl0.where(dfTl0.CD == 1).dropna()
dfTl0unload = dfTl0.where(dfTl0.CD == 0).dropna()

# Statistics 
* taking code from Laret Statistics_1.ipyb notebook

## T-Statistic

#### All Full Dataset

In [19]:
stats.ttest_ind(snowunloaddf['temp'], snowstaydf['temp'])

Ttest_indResult(statistic=7.594783821694393, pvalue=1.0005378370793078e-13)

In [20]:
stats.ttest_ind(snowunloaddf['shortwave'], snowstaydf['shortwave'])

Ttest_indResult(statistic=16.056309181851557, pvalue=1.4740720436900322e-49)

In [21]:
stats.ttest_ind(snowunloaddf['windspeed'], snowstaydf['windspeed'])

Ttest_indResult(statistic=10.1884610727472, pvalue=8.232606762441789e-23)

### Shortwave Thresholded Data
**SW > 600**

In [22]:
stats.ttest_ind(dfSW600stay['temp'], dfSW600unload['temp'])

Ttest_indResult(statistic=-3.9832554591177463, pvalue=0.00014895004522626163)

In [23]:
stats.ttest_ind(dfSW600stay['windspeed'], dfSW600unload['windspeed'])

Ttest_indResult(statistic=-1.1560381890134284, pvalue=0.2511062975873969)

**400 < SW < 600**

In [24]:
stats.ttest_ind(dfSW400600stay['temp'], dfSW400600unload['temp'])

Ttest_indResult(statistic=-1.5890557714408284, pvalue=0.11649127739605475)

In [25]:
stats.ttest_ind(dfSW400600stay['windspeed'], dfSW400600unload['windspeed'])

Ttest_indResult(statistic=-1.2518482174535521, pvalue=0.21473365956192184)

**SW < 400**

In [26]:
stats.ttest_ind(dfSW400stay['temp'], dfSW400unload['temp'])

Ttest_indResult(statistic=-3.0642578274204793, pvalue=0.0022914838721290872)

In [27]:
stats.ttest_ind(dfSW400stay['windspeed'], dfSW400unload['windspeed'])

Ttest_indResult(statistic=-6.960955393857682, pvalue=9.876478843150137e-12)

### Temperature Thresholded Data 
**T>0**

In [28]:
stats.ttest_ind(dfTg0stay['shortwave'], dfTg0unload['shortwave'])

Ttest_indResult(statistic=-3.5815874139892365, pvalue=0.0007151629436889473)

In [29]:
stats.ttest_ind(dfTg0stay['windspeed'], dfTg0unload['windspeed'])

Ttest_indResult(statistic=3.248493244693405, pvalue=0.0019635607449520454)

**T<0**

In [30]:
stats.ttest_ind(dfTl0stay['shortwave'], dfTl0unload['shortwave'])

Ttest_indResult(statistic=-12.776329341984717, pvalue=1.9798780328604836e-33)

In [31]:
stats.ttest_ind(dfTl0stay['windspeed'], dfTl0unload['windspeed'])

Ttest_indResult(statistic=-12.113935036606446, pvalue=1.558783481796821e-30)

_____________________________

## Z-Score, continued
* Simplify this into one block of code to run for all the other subsets of data 

#### Shortwave Thresholded Data 

In [39]:
# The full dataset 
var1 = snowstaydf['temp']
var2 = snowunloaddf['temp']

# var1 = snowstaydf['shortwave']
# var2 = snowunloaddf['shortwave']

# var1 = snowstaydf['windspeed']
# var2 = snowunloaddf['windspeed']

In [40]:
# Check that we have a large enough sample size (n>30)
n = len(var1)
print("Snow Stay Sample Size: ", n)
m = len(var2)
print("Snow Unload Sample Size: ", m)

# We want alpha to be 0.05 
alpha = 0.05 

# Which gives a confidence of 0.95
conf = 1 - alpha 

# Determine which value in the z-distribution corresponds to the 0.95 confidence in the CDF
z_alpha = stats.norm.ppf(conf)
print("z_alpha = {}".format(z_alpha))

# Compuate the pooled standard deviation 
pooled_sd = np.sqrt(var1.std(ddof=1)**2 / n + var2.std(ddof=1)**2 / m )

# Hypothesizing no change
mu_0 = 0

# Compute z-score
zscore = (var1.mean() - var2.mean() - mu_0)/pooled_sd
print("z-score = {}".format( np.round(zscore,3) )) 

# Compute p-value from z-score
pvalue = 1 - stats.norm.cdf(zscore)
print("p = {}".format( np.round(pvalue,3)))

Snow Stay Sample Size:  465
Snow Unload Sample Size:  230
z_alpha = 1.6448536269514722
z-score = -7.679
p = 1.0


In [47]:
# # For SW thresholds 
# # SW > 600
# var1 = dfSW600stay['temp']
# var2 = dfSW600unload['temp']
# var1 = dfSW600stay['windspeed']
# var2 = dfSW600unload['windspeed']

# # 400 < SW < 600
# var1 = dfSW400600stay['temp']
# var2 = dfSW400600unload['temp']
# var1 = dfSW400600stay['windspeed']
# var2 = dfSW400600unload['windspeed']

# # SW < 400
var1 = dfSW400stay['temp']
var2 = dfSW400unload['temp']
# var1 = dfSW400stay['windspeed']
# var2 = dfSW400unload['windspeed']

In [48]:
# Check that we have a large enough sample size (n>30)
n = len(var1)
print("Snow Stay Sample Size: ", n)
m = len(var2)
print("Snow Unload Sample Size: ", m)

# We want alpha to be 0.05 
alpha = 0.05 

# Which gives a confidence of 0.95
conf = 1 - alpha 

# Determine which value in the z-distribution corresponds to the 0.95 confidence in the CDF
z_alpha = stats.norm.ppf(conf)
print("z_alpha = {}".format(z_alpha))

# Compuate the pooled standard deviation 
pooled_sd = np.sqrt(var1.std(ddof=1)**2 / n + var2.std(ddof=1)**2 / m )

# Hypothesizing no change
mu_0 = 0

# Compute z-score
zscore = (var1.mean() - var2.mean() - mu_0)/pooled_sd
print("z-score = {}".format( np.round(zscore,3) )) 

# Compute p-value from z-score
pvalue = 1 - stats.norm.cdf(zscore)
print("p = {}".format( np.round(pvalue,3)))

Snow Stay Sample Size:  427
Snow Unload Sample Size:  113
z_alpha = 1.6448536269514722
z-score = -3.135
p = 0.999


#### Temperature Thresholded Data

In [51]:
# For temperature thresholds 
# var1 = dfTg0stay['windspeed']
# var2 = dfTg0unload['windspeed']
# var1 = dfTg0stay['shortwave']
# var2 = dfTg0unload['shortwave']

var1 = dfTl0stay['windspeed']
var2 = dfTl0unload['windspeed']
# var1 = dfTl0stay['shortwave']
# var2 = dfTl0unload['shortwave']

In [52]:
# Check that we have a large enough sample size (n>30)
n = len(var1)
print("Snow Stay Sample Size: ", n)
m = len(var2)
print("Snow Unload Sample Size: ", m)

# We want alpha to be 0.05 
alpha = 0.05 

# Which gives a confidence of 0.95
conf = 1 - alpha 

# Determine which value in the z-distribution corresponds to the 0.95 confidence in the CDF
z_alpha = stats.norm.ppf(conf)
print("z_alpha = {}".format(z_alpha))

# Compuate the pooled standard deviation 
pooled_sd = np.sqrt(var1.std(ddof=1)**2 / n + var2.std(ddof=1)**2 / m )

# Hypothesizing no change
mu_0 = 0

# Compute z-score
zscore = (var1.mean() - var2.mean() - mu_0)/pooled_sd
print("z-score = {}".format( np.round(zscore,3) )) 

# Compute p-value from z-score
pvalue = 1 - stats.norm.cdf(zscore)
pvalue2 = stats.norm.cdf(zscore)
print("p = {}".format( np.round(pvalue,3)))
print("p = {}".format( np.round(pvalue2,3)))

Snow Stay Sample Size:  458
Snow Unload Sample Size:  179
z_alpha = 1.6448536269514722
z-score = -11.151
p = 1.0
p = 0.0
